In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


################################################################################
## Music Stuff
################################################################################
from masterdb import masterdb
from musicUtils import *
from myMusicDBMap import myMusicDBMap


################################################################################
## Pandas Stuff
################################################################################
import sys
from collections import Counter
print("Python: {0}".format(sys.version))


################################################################################
## Prints
################################################################################
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-06-22 22:12:33.882335


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# My Music DataBase

In [2]:
mdb = myMusicDBMap(debug=False)
mdb.getKnownDBData()

=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: True
    Found 3909 ID -> Name entries
    Found 3908 Name -> ID entries
    Found 48662 Albums
=========================   allmusic   =========================
Summary Statistics For DB: AllMusic
    Using Known Artists: True
    Found 3839 ID -> Name entries
    Found 3836 Name -> ID entries
    Found 27759 Albums
=========================   musicbrainz   =========================
Summary Statistics For DB: MusicBrainz
    Using Known Artists: True
    Found 3573 ID -> Name entries
    Found 3572 Name -> ID entries
    Found 26401 Albums
=========================   acebootlegs   =========================
Summary Statistics For DB: AceBootlegs
    Using Known Artists: True
    Found 144 ID -> Name entries
    Found 144 Name -> ID entries
    Found 2000 Albums
=========================   rateyourmusic   =========================
Summary Statistics For DB: RateYour

In [81]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase(debug=True)
mmb.findArtistAlbums(count=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Piggy/Music/Matched']
Current Time is Thu Jun 25, 2020 20:30:35 for Finding All Artist Albums
Current Time is Thu Jun 25, 2020 20:30:35 for =====> A
Current Time is Thu Jun 25, 2020 20:30:38 for Done with =====> A
Process [Done with =====> A] took 3 seconds.

Current Time is Thu Jun 25, 2020 20:30:38 for =====> B
Current Time is Thu Jun 25, 2020 20:30:42 for Done with =====> B
Process [Done with =====> B] took 3 seconds.

Current Time is Thu Jun 25, 2020 20:30:42 for =====> C
Current Time is Thu Jun 25, 2020 20:30:46 for Done with =====> C
Process [Done with =====> C] took 4 seconds.

Current Time is Thu Jun 25, 2020 20:30:46 for =====> D
Current Time is Thu Jun 25, 2020 20:30:52 for Done with =====> D
Process [Done with =====> D] took 5 seconds.

Current Time is Thu Jun 25, 2020 20:30:52 for =====> E
Current Time is Thu Jun 25, 2020 20:30:55 for Done with =====> E
Process 

In [82]:
df = mmb.getDF()
df

,BoxSet,Bootleg,Mix,Media,Unknown,Random,Todo,Match,UnMatched,Files,Prime
A Flock Of Seagulls,0,0,0,0,0,0,0,26,1,None,A
A Global Threat,0,0,0,0,0,0,0,4,0,None,A
A Great Big World,0,0,0,0,0,1,0,0,1,None,A
A Tribe Called Quest,0,0,0,0,0,0,2,14,0,None,A
A*Teens,0,0,0,0,0,0,1,1,0,None,A
...,...,...,...,...,...,...,...,...,...,...,...
Zen Guerrilla,0,0,0,0,0,0,0,1,0,None,Z
Zero Zero,0,0,0,0,0,0,0,1,0,None,Z
Zipgun,0,0,0,0,0,0,0,1,0,None,Z
Zoobombs,0,0,0,0,0,0,0,2,0,None,Z


In [87]:
sum([x for x in df.sum() if isinstance(x, int)])

21490

In [18]:
for prime,primeDF in df[df["Match"] == 0].groupby("Prime"):
    subDF = primeDF[primeDF["UnMatched"] == 0]
    if subDF.shape[0] > 0:
        print('='*50)
        print(prime)
        print(subDF)
        print("\n\n")

A
             BoxSet  Bootleg  Mix  Media  Unknown  Random  Todo  Match  UnMatched Prime
Al B. Sure!       0        0    0      0        0       1     0      0          0     A
Ann Lee           0        0    0      0        0       0     2      0          0     A
Asylum Pyre       0        0    0      0        0       0     4      0          0     A



B
        BoxSet  Bootleg  Mix  Media  Unknown  Random  Todo  Match  UnMatched Prime
B.B.E.       0        0    0      0        0       0     1      0          0     B
Bambee       0        0    0      0        0       0     6      0          0     B



G
                BoxSet  Bootleg  Mix  Media  Unknown  Random  Todo  Match  UnMatched Prime
George Clinton       0        0    0      0        0       5     0      0          0     G





In [ ]:
matched

In [ ]:
dirval = Series(df.index).apply(mmb.getPrimeDirectory)
dirval
#df["Prime"] = dirval

# Find Unmatched Data

In [ ]:
artistName = ""
db = "AllMusic"
unMatchedAlbums = mmb.getArtistAlbumsByArtist(artistName).getUnmatched()
artistNameDBIDs = mdb.getArtistDBIDs(artistName, db)

In [ ]:
unMatchedAlbums

#### Show None Null DB Matches

In [ ]:
artistName = "Tim Reynolds"
dbMatches = {k: v for k,v in mdb.getArtistDataIDs(artistName).items() if v is not None}
dbMatches

In [ ]:
for db,dbID in dbMatches.items():
    print("="*50)
    print(db,'\t--->',dbID)

    artistNameDBIDs = mdb.getArtistDBIDs(artistName, db)
    dbData = mdb.getArtistAlbumsFromID(db, dbID)
    for mediaType,mediaData in dbData.items():
        print('\t',mediaType)
        print('\t\t',list(mediaData.values()))
    print("\n")

In [ ]:
disc = discogs("discogs")
mymdb = masterdb("discogs", disc, force=False)
mymdb.setMyMusicDB(mdb)
slimArtistDB = mymdb.
slimArtistDB

## Change Music DB

In [ ]:
mdb.rmArtistDBKey("Alejandro Sanz", "Discogs")

In [ ]:
mdb.add("Alejandro Sanz", "Discogs", "217655")

In [ ]:
mdb.save()

In [ ]:
from timeUtils import clock, elapsed
from myMusicDBMap import myMusicDBMap

class myMusicAnalysis:
    def __init__(self, mdb, debug=False):
        print("=================== Analysis of My Music ===================")
        self.debug = debug
        self.mdb   = mdb
        self.mdb.show()
        
        self.df = None
        self.results = {}

        
    def getArtistUnMatchedAlbums(self):
        retval         = {}


        ######################################################################
        #### Loop Over Prime Directories
        ######################################################################
        for primeDir in getPrimeDirectories():
            artistPrimeDirMap = getArtistPrimeDirMap(primeDir)
            print("UnMatched Albums: {0}".format(primeDir))
            #if primeDir.startswith("B"):
            #    break


            ######################################################################
            #### Loop Over Artist Name <-> Prime Map Items
            ######################################################################
            for artistName, artistPrimeDirs in artistPrimeDirMap.items():
                retval[artistName] = {"MyAlbums": [], "Todo": [], "Unknown": [], "Random": [], "Volumes": artistPrimeDirs}
                myAlbums           = {}
                for dirval in artistPrimeDirs:
                    myAlbums["MyAlbums"] = getMyMusicAlbums(dirval, returnNames=True)
                    myAlbums["Todo"]     = getMyTodoMusicAlbums(dirval)
                    myAlbums["Unknown"]  = getMyUnknownMusicAlbums(dirval)
                    myAlbums["Random"]   = getMyRandomMusic(dirval)

                    for key in myAlbums.keys():
                        retval[artistName][key] += myAlbums[key]

                for key in retval[artistName].keys():
                    retval[artistName][key] = len(retval[artistName][key])


        return retval



    def getArtistMatchedAlbums(self):
        albumTypes     = getAlbumTypes(keys=True)
        retval         = {}


        ######################################################################
        #### Loop Over Prime Directories
        ######################################################################
        for primeDir in getPrimeDirectories():
            artistPrimeDirMap = getArtistPrimeDirMap(primeDir) 
            print("Matched Albums: {0}".format(primeDir))
            #if primeDir.startswith("B"):
            #    break




            ######################################################################
            #### Loop Over Artist Name <-> Prime Map Items
            ######################################################################
            for artistName, artistPrimeDirs in artistPrimeDirMap.items():


                ######################################################################
                #### Get My Albums
                ######################################################################
                albumTypeResults = {}
                artistMatchDirs = [setDir(artistPrimeDir, 'Match', forceExist=False) for artistPrimeDir in artistPrimeDirs if isDir(artistPrimeDir, 'Match')]
                for albumType in albumTypes:
                    albumTypeResults[albumType] = []
                    for db in self.mdb.getDBs():
                        albumDBTypeNames = getAlbumTypes(db, albumType)
                        for albumDBTypeName in albumDBTypeNames:
                            for artistMatchDir in artistMatchDirs:
                                albumDBTypeDir = setDir(artistMatchDir, albumDBTypeName, forceExist=False)
                                if isDir(albumDBTypeDir):
                                    #print(albumDBTypeDir)
                                    albumTypeResults[albumType] += [getDirBasics(x)[-1] for x in findDirs(albumDBTypeDir)]
                                    #print(len(albumTypeResults),'\t',albumTypeResults)
                    albumTypeResults[albumType] = len(list(set(albumTypeResults[albumType])))
                retval[artistName] = albumTypeResults


        return retval
    
    
    def analyze(self):
        start, cmt = clock("Analyzing Artist Albums")

        artistUnMatchedAlbums = self.getArtistUnMatchedAlbums()
        artistMatchedAlbums   = self.getArtistMatchedAlbums()    

        self.results   = {}
        artistNames = set(list(artistUnMatchedAlbums.keys()) + list(artistMatchedAlbums.keys()))
        for artistName in sorted(artistNames):
            artistMatchedAlbumResults   = artistMatchedAlbums.get(artistName)
            artistUnMatchedAlbumResults = artistUnMatchedAlbums.get(artistName)
            #print(artistName,'\t',artistMatchedAlbumResults,'\t',artistUnMatchedAlbumResults)

            self.results[artistName]          = artistUnMatchedAlbumResults
            self.results[artistName]["Match"] = artistMatchedAlbumResults
            
        
        self.df = DataFrame(self.results).T            
            
        elapsed(start, cmt)
            
            
    def getDataFrame(self):
        if df is None:
            raise ValueError("DataFrame is not calculated!")
        return self.df
    
    
    def getResults(self):
        return self.results

## Merge Biggy and Music

In [ ]:
biggy = "/Volumes/Piggy/Music/Matched_Biggy"
music = "/Volumes/Piggy/Music/Matched_Music"

In [ ]:
test = False

from fsUtils import isDir, setSubDir, moveDir
from fileUtils import getDirBasics

for biggyDirVal in glob("{0}/*".format(biggy)):
    if not isDir(biggyDirVal):
        continue
        
    primeVal    = getDirBasics(biggyDirVal)[-1]
    musicDirVal = setDir(music, primeVal)
    
    for biggyArtistDirVal in glob("{0}/*".format(biggyDirVal)):
        if not isDir(biggyArtistDirVal):
            continue

        artistVal         = getDirBasics(biggyArtistDirVal)[-1]
        musicArtistDirVal = setDir(musicDirVal, artistVal)
        #print(primeVal,artistVal,'\t',isDir(musicArtistDirVal),isDir(biggyArtistDirVal))
        #print(musicArtistDirVal)
    
        ### Check For Music Artist and Move if not found
        if not isDir(musicArtistDirVal):
            srcDir  = biggyArtistDirVal
            dstDir  = musicArtistDirVal
            print("Moving {0} to {1}".format(artistVal, music))
            if test is False:
                moveDir(srcDir, dstDir, debug=True)
            continue
        else:
            pass
            print("Found {0}".format(artistVal))
            
            
            
            
        ### Loop over artist albums directories since we've found multiple copies
        for biggyArtistAlbumDirVal in glob("{0}/*".format(biggyArtistDirVal)):
            if not isDir(biggyArtistAlbumDirVal):
                continue

            artistAlbumVal         = getDirBasics(biggyArtistAlbumDirVal)[-1]
            musicArtistAlbumDirVal = setDir(musicArtistDirVal, artistAlbumVal)
    
            ### Check For Music ArtistAlbum and Move if not found
            if not isDir(musicArtistAlbumDirVal):
                srcDir  = biggyArtistAlbumDirVal
                dstDir  = musicArtistAlbumDirVal
                print("   Moving {0}/{1} to {2}".format(artistVal, artistAlbumVal, music))
                if test is False:
                    moveDir(srcDir, dstDir, debug=True)
                continue
            else:
                print("   Found {0}/{1}".format(artistVal, artistAlbumVal))
            
            
            
            
            ### Loop over artist albums sub directories since we've found multiple copies
            for biggyArtistAlbumSubDirVal in glob("{0}/*".format(biggyArtistAlbumDirVal)):
                if not isDir(biggyArtistAlbumSubDirVal):
                    continue

                artistAlbumSubVal         = getDirBasics(biggyArtistAlbumSubDirVal)[-1]
                musicArtistAlbumSubDirVal = setDir(musicArtistAlbumDirVal, artistAlbumSubVal)

                ### Check For Music ArtistAlbum and Move if not found
                if not isDir(musicArtistAlbumSubDirVal):
                    srcDir  = biggyArtistAlbumSubDirVal
                    dstDir  = musicArtistAlbumSubDirVal
                    print("      Moving {0}/{1}/{2} to {3}".format(artistVal, artistAlbumVal, artistAlbumSubVal, music))
                    if test is False:
                        moveDir(srcDir, dstDir, debug=True)
                    continue
                else:
                    print("      Found {0}/{1}/{2}".format(artistVal, artistAlbumVal, artistAlbumSubVal))
            
            


                ### Loop over artist albums sub directories since we've found multiple copies
                for biggyArtistAlbumSubSubDirVal in glob("{0}/*".format(biggyArtistAlbumSubDirVal)):
                    if not isDir(biggyArtistAlbumSubSubDirVal):
                        continue

                    artistAlbumSubSubVal         = getDirBasics(biggyArtistAlbumSubSubDirVal)[-1]
                    musicArtistAlbumSubSubDirVal = setDir(musicArtistAlbumSubDirVal, artistAlbumSubSubVal)

                    ### Check For Music ArtistAlbum and Move if not found
                    if not isDir(musicArtistAlbumSubSubDirVal):
                        srcDir  = biggyArtistAlbumSubSubDirVal
                        dstDir  = musicArtistAlbumSubSubDirVal
                        print("         Moving {0}/{1}/{2}/{3} to {4}".format(artistVal, artistAlbumVal, artistAlbumSubVal, artistAlbumSubSubVal, music))
                        if test is False:
                            moveDir(srcDir, dstDir, debug=True)
                        continue
                    else:
                        print("         Found {0}/{1}/{2}/{3}".format(artistVal, artistAlbumVal, artistAlbumSubVal, artistAlbumSubSubVal))

In [ ]:
import string
from fsUtils import isDir, setDir
from listUtils import getFlatList
from timeUtils import clock, elapsed
from unicodedata import normalize
from searchUtils import findDirs, findAll, findNearest
from fileUtils import getDirBasics, getBaseFilename
from glob import glob
from os.path import join
from pandas import DataFrame, Series
from collections import Counter

from primeDirectory import primeDirectory

class myArtistAlbumType:
    def __init__(self, debug=False):
        self.albums = {}
        
    def getNum(self):
        num = sum(len(x) for x in self.albums.values())
        return num
    
    def setAlbums(self, albums):
        self.albums = albums
        
    def getAlbums(self):
        return self.albums
    
    def getNumAlbums(self):
        return self.getNum()

In [ ]:
maa = myArtistAlbums("Dave Matthews")
print(maa.dirvals)
print(maa.myMusicDirs)

In [ ]:

                                            
    def getDirVals(self):
        return self.dirvals.keys()
        
        
        
    def setSubDirVals(self, artistAlbumType):
        dirvals = {}
        for dirval,albums in artistAlbumType.getAlbums().items():
            dirvals[dirval] = albums
        return dirvals
            
    def organize(self):
        for dirval,dirAlbums in self.dirvals.items():
            for albumType,albums in dirAlbums.items():
                self.dirvals[dirval] = self.setSubDirVals(self.unmatched)
        
        for dirval,albums in self.matched.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Matched"] = albums
        
        for dirval,albums in self.todo.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Todo"] = albums
        
        for dirval,albums in self.todo.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Todo"] = albums
        
        for dirval,albums in self.todo.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Todo"] = albums
        
        for dirval,albums in self.todo.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Todo"] = albums
        
        for dirval,albums in self.unknown.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Unknown"] = albums
        
        for dirval,albums in self.random.items():
            if self.dirvals.get(dirval) is None:
                self.dirvals[dirval] = {}
            self.dirvals[dirval]["Random"] = albums
            
    

    def getNumVolumes(self):
        oaa = self.getOrganizedAlbums()
        volumes = {volume: sum([len(y) for y in x.values()]) for volume, x in oaa.items()}
        return volumes

    
    def setUnmatched(self, unmatched):
        self.unmatched = unmatched
    def getUnmatched(self):
        return self.unmatched
    def getNumUnmatched(self):
        return self.getNum(self.unmatched)
    

    def setMatched(self, matched):
        self.matched = matched
    def getMatched(self):
        return self.matched
    def getNumMatched(self):
        return self.getNum(self.matched)
    

    def setTodo(self, todo):
        self.todo = todo
    def getTodo(self):
        return self.todo
    def getNumTodo(self):
        return self.getNum(self.todo)


    def setUnknown(self, unknown):
        self.unknown = unknown
    def getUnknown(self):
        return self.unknown
    def getNumUnknown(self):
        return self.getNum(self.unknown)


    def setRandom(self, random):
        self.random = random
    def getRandom(self):
        return self.random
    def getNumRandom(self):
        return self.getNum(self.random)

        
        self.unmatched = myArtistAlbumType()
        self.matched   = myArtistAlbumType()
        self.boxset    = myArtistAlbumType()
        self.bootleg   = myArtistAlbumType()
        self.mix       = myArtistAlbumType()
        self.media     = myArtistAlbumType()
        self.todo      = myArtistAlbumType()
        self.unknown   = myArtistAlbumType()
        self.random    = myArtistAlbumType()
        
        
            
    def getOrganizedAlbums(self):
        self.organize()
        return self.dirvals
        

    def getDict(self):
            
        retval = {"Unmatched": self.getNumUnmatched(),
                  "Matched":   self.getNumMatched(),
                  "Todo":      self.getNumTodo(),
                  "Unknown":   self.getNumUnknown(),
                  "Random":    self.getNumRandom()}
        return retval
    
        
    def info(self):
        print("Artist Albums For {0}".format(self.artistName))
        print("  Unmatched: {0}".format(self.unmatched))
        print("  Matched:   {0}".format(self.matched))
        print("  Todo:      {0}".format(self.todo))
        print("  Unknown:   {0}".format(self.unknown))
        print("  Random:    {0}".format(self.random))

        

In [ ]:
mmb = myMusicBase(debug=True)
mmb.findArtistAlbums()

In [ ]:
df = mmb.getDF()
df

In [ ]:


    
    

    
    
    
    ###################################################################################################
    # My Music Directories
    ###################################################################################################    
    def getMyUnmatchedAlbums(self, dirval, returnNames=False):    
        myMusicAlbums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in self.myMusicDirs]
        if returnNames is True:
            myMusicUnmatched = [getDirBasics(x)[-1] for x in myMusicAlbums]
        return myMusicUnmatched


    def getMyMatchedMusicAlbums(self, dirval, byKey=False):  
        matchval = join(dirval, self.matchDir, "*")
        matchedAlbums = []
        for dname in glob(matchval):
            matchedAlbums += [getDirBasics(x)[-1].split(" :: ")[0] for x in findDirs(dname)]
        return matchedAlbums


    def getMyTodoMusicAlbums(self, dirval):
        todoAlbums = []
        for dval in self.todoDirs:
            todoval = join(dirval, dval)
            for dname in glob(todoval):
                todoAlbums += [getDirBasics(x)[-1] for x in findDirs(dname)]
        return todoAlbums


    def getMyUnknownMusicAlbums(self, dirval):
        todoAlbums = []
        for dval in self.unknownDirs:
            todoval = join(dirval, dval)
            for dname in glob(todoval):
                todoAlbums += [getDirBasics(x)[-1] for x in findDirs(dname)]
        return todoAlbums


    def getMyRandomMusic(self, dirval):
        randomMusic = []
        for dval in self.randomDirs:
            todoval = join(dirval, dval)
            for dname in glob(todoval):
                randomMusic += [getBaseFilename(x) for x in findAll(dname)]
        return randomMusic


    
    def getArtists(self):
        return list(self.artistAlbums.keys())
    
    
    def getArtistAlbums(self):
        return self.artistAlbums


    def getArtistAlbumsByArtist(self, artistName):
        return self.artistAlbums.get(artistName)
        

    def getDictByArtist(self, artistName):
        mma = self.getArtistAlbumsByArtist(artistName)
        if mma is not None:
            return mma.getDict()
        return {}
    
    
    def getDataFrame(self):
        tmp    = {artistName: artistData.getDict() for artistName, artistData in self.getArtistAlbums().items()}
        df     = DataFrame(tmp).T
        dirval = Series(df.index).apply(self.getPrimeDirectory)
        dirval.index = df.index
        df["Prime"] = dirval
        return df
    def getDF(self):
        return self.getDataFrame()


    
    def findArtistAlbums(self):
        start, cmt = clock("Finding All Artist Albums")
        
        self.artistAlbums = {}
        for primeDir in self.getPrimeDirectories():
            if self.debug:
                startPrime, cmtPrime = clock("=====> {0}".format(primeDir))
            artistPrimeDirItems = self.getArtistPrimeDirMap(primeDir)
            for artistName, artistPrimeDirs in artistPrimeDirItems.items():
                maa = myArtistAlbums(artistName, artistPrimeDirs)

                ######################################################################
                #### Get My Music Albums
                ######################################################################
                myUnmatchedAlbums = {dirval: self.getMyUnmatchedAlbums(dirval, returnNames=True) for dirval in artistPrimeDirs}
                maa.unmatched.setAlbums(myUnmatchedAlbums)


                ######################################################################
                #### Get My Matched Albums
                ######################################################################
                myMatchedAlbums = {dirval: self.getMyMatchedMusicAlbums(dirval) for dirval in artistPrimeDirs}
                maa.matched.setAlbums(myMatchedAlbums)


                ######################################################################
                #### Get My Unknown Albums
                ######################################################################
                myUnknownAlbums = {dirval: self.getMyUnknownMusicAlbums(dirval) for dirval in artistPrimeDirs}
                maa.unknown.setAlbums(myUnknownAlbums)


                ######################################################################
                #### Get My Todo Albums
                ######################################################################
                myTodoAlbums = {dirval: self.getMyTodoMusicAlbums(dirval) for dirval in artistPrimeDirs}
                maa.todo.setAlbums(myTodoAlbums)


                ######################################################################
                #### Get My Random Music
                ######################################################################
                myRandomMusic = {dirval: self.getMyRandomMusic(dirval) for dirval in artistPrimeDirs}
                maa.random.setAlbums(myRandomMusic)
                
                
                self.artistAlbums[artistName] = maa
                
            if self.debug:
                elapsed(startPrime, cmtPrime)
                print("")
                
        elapsed(start, cmt)
        
        
    def show(self):
        minMyAlbums   = 2
        minTodoAlbums = 5
        downloadCut   = 5

        ranks = {"Matches": Counter(), "Albums": Counter()}

        print("{0: <35}| {1: <10}| {2: <10}| {3: <10}| {4: <10}| {5: <10}| {6: <10}| {7: <12}| {8: <12}|".format("Artist", "Volumes", "Matches", "# Albums", "Todo", "Unknown", "Random", "No Match", "Many Albums"))
        print("{0: <35}| {1: <10}| {2: <10}| {3: <10}| {4: <10}| {5: <10}| {6: <10}| {7: <12}| {8: <12}|".format("------", "-------", "-------", "--------", "----", "-------", "------", "--------", "-----------"))

        for artistName, artistData in self.mmb.getArtistAlbums().items():
            
            ### Artist Name
            print("{0: <35}".format(artistName), end="")

            ### Volumes
            oaa     = artistData.getOrganizedAlbums()
            volumes = {volume: sum([len(y) for y in x.values()]) for volume, x in oaa.items()}
            print("| {0: <10}".format(self.printAlbums(volumes.values())), end="")

            ### Matched Albums
            numMatched = artistData.getNumMatched()
            print("| {0: <10}".format(numMatched), end="")

            ### Unmatched Albums
            numUnmatched = artistData.getNumUnmatched()
            print("| {0: <10}".format(numUnmatched), end="")

            ### Todo Albums
            numTodo = artistData.getNumTodo()
            print("| {0: <10}".format(numTodo), end="")

            ### Unknown Albums
            numUnknown = artistData.getNumUnknown()
            print("| {0: <10}".format(numUnknown), end="")

            ### Random Albums
            numRandom = artistData.getNumRandom()
            print("| {0: <10}".format(numRandom), end="")

            
            ### Check For No Match
            noMatchText = "{0}..".format(artistName[:9])
            if numMatched > 0:
                noMatchText = ""
            print("| {0: <12}".format(noMatchText), end="")

            
            ### Check For Many Albums
            manyAlbumsText = "{0}..".format(artistName[:9])
            if numUnmatched < 3:
                manyAlbumsText = ""
            print("| {0: <12}".format(manyAlbumsText), end="")

            ### Return
            print("|")